**Connect Google Drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

# **1) Clone the Darknet**

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

# **2) Compile Darknet using Nvidia GPU**

In [ ]:
# Change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

# **3) Configure Darknet network for training YOLO V3**

In [ ]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg

# **3) Create .names and .data files**

In [ ]:
!echo 'dot\crack' > data/obj.names
!echo -e 'classes= 2\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/custom_object_detection_yolo' > data/obj.data

# **4) Save yolov3.training.cfg and obj.names files in Google Drive**

In [ ]:
!cp cfg/yolov3_training.cfg /mydrive/custom_object_detection_yolo/yolov3_testing.cfg
!cp data/obj.names /mydrive/custom_object_detection_yolo/classes.txt

# **5) Download pre-trained weights for the convolutional layers file**

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

# **6) Create a folder and unzip the image dataset**
**The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "custom_object_detection_yolo" on Google Drive**

In [ ]:
!mkdir data/obj
!unzip /mydrive/custom_object_detection_yolo/yolo_images.zip -d data/obj

**We're going to convert the class index on the .txt files. As we're working with two class, it's supposed to be class 0 and 1. If the index is different from 0 or 1 then we're going to change it.**

In [ ]:
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            if numbers:

                # Define coordinates
                if numbers[0] == '15':
                  text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                elif numbers[0] == '16':
                  text = "{} {} {} {} {}".format(1, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

## **7) Create train.txt file**

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")

file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

# **8) Start the training**

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
#Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /content/gdrive/MyDrive/custom_object_detection_yolo/yolov3_training_last.weights -dont_show